In [1]:
import pandas as pd

In [2]:
#SIFTS database from http://www.ebi.ac.uk
## 2019/06/23 - 00:46 | PDB: 25.19 | UniProt: 2019.06
#A summary of the PDBe to UniProt residue level mapping, 
#showing the start and end residues of the mapping using SEQRES, PDB sequence and UniProt numbering.
pdbChainUniprot = pd.read_csv("/home/yuan/Documents/pdb_chain_uniprot.csv")

In [3]:
pdbChainUniprot.iloc[0:10,:]

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,1ao7,A,P01892,1,275,1,None,25,299
1,1ao7,B,P61769,2,100,1,99,21,119
2,1ao7,C,P14079,1,9,1,9,11,19
3,1ao7,D,A0A075B6T6,1,90,1,92,23,112
4,1ao7,D,P01848,112,204,118,None,1,93
5,1ao7,E,A0A0K0K1A5,1,95,None,96,20,114
6,1ao7,E,A0A5B9,117,245,118,246,1,129
7,1axk,A,P23904,1,156,1,156,82,237
8,1axk,A,P18429,157,341,157,341,29,213
9,1axk,B,P23904,1,156,1,156,82,237


In [4]:
validRow = (pdbChainUniprot["PDB_BEG"].str.isnumeric()) & (pdbChainUniprot["PDB_END"].str.isnumeric()) 
pdbChainIntegrity = pdbChainUniprot[validRow]

pdbChainIntegrity.iloc[0:10,:]

    ### SEQRES, PDB, UniProt numbering 


### Use the table above to return PDB info if available

In [5]:
#If absense in the table, empty dataframe returned
def IsPDBInfoAvailable(UniprotAccession, Position):
    rowWithAccession = pdbChainIntegrity[pdbChainIntegrity["SP_PRIMARY"].isin([UniprotAccession])]
    if(rowWithAccession.shape[0]>0):
        rowWithAccession = rowWithAccession[(rowWithAccession["SP_BEG"]<Position)
                                            & (rowWithAccession["SP_END"]>Position)]
        rowWithAccession
        rowWithAccession.loc[:,"Ksites"] = Position + (rowWithAccession["PDB_BEG"].astype(int)
                                                       - rowWithAccession["SP_BEG"].astype(int))
        return rowWithAccession
    return pdbChainUniprot[0:0]

In [6]:
PTM_sites = pd.read_csv("/home/yuan/Documents/uniprot_to_pdb_human_pos_neg.csv")

In [7]:
IsPDBInfoAvailable("P24182",27)

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites
114857,3g8c,A,P24182,1,444,1,444,1,444,27
114858,3g8c,B,P24182,1,444,1,444,1,444,27
114859,3g8d,A,P24182,1,444,1,444,1,444,27
114860,3g8d,B,P24182,1,444,1,444,1,444,27
132335,3jzf,A,P24182,22,470,1,449,1,449,27


In [24]:
result = []
for i in range(0,PTM_sites.shape[0]):
    oneAccession = IsPDBInfoAvailable(PTM_sites.iloc[i,0],PTM_sites.iloc[i,1])
    if oneAccession.shape[0]>0:
        oneAccession["class"] = PTM_sites.iloc[i,2]
        oneAccession["k_site_uniprot"] = PTM_sites.iloc[i,1]
        result.append(oneAccession)
result = pd.concat(result)

### Verification: 3g8d: 27th and 124th are K residue

In [31]:
result.to_csv("/home/yuan/Documents/pdb_chain_ksites_huamn_pos_neg_in_paper.csv",index=False)

In [20]:
result["PDB"].unique().size

1640

In [26]:
result.iloc[0:5,]

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites,class,k_site_uniprot
185659,4crw,A,A5YKK6,7,231,1093,1317,1093,1317,1227,n,1227
197753,4gmj,A,A5YKK6,5,229,1093,1317,1093,1317,1227,n,1227
197755,4gmj,C,A5YKK6,5,229,1093,1317,1093,1317,1227,n,1227
197757,4gmj,E,A5YKK6,5,229,1093,1317,1093,1317,1227,n,1227
197761,4gml,A,A5YKK6,11,235,1093,1317,1093,1317,1227,n,1227


In [32]:
result.groupby(by=["SP_PRIMARY","k_site_uniprot","class"]).head(1)

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites,class,k_site_uniprot
185659,4crw,A,A5YKK6,7,231,1093,1317,1093,1317,1227,n,1227
73067,2pkt,A,O00151,2,88,1,87,1,87,71,p,71
65071,2lkn,A,O00170,1,165,2,166,2,166,112,p,112
177610,4aif,A,O00170,1,144,172,315,172,315,273,p,273
319196,5l4k,Q,O00231,1,422,1,422,1,422,185,p,185
319196,5l4k,Q,O00231,1,422,1,422,1,422,246,n,246
319195,5l4k,P,O00232,1,456,1,456,1,456,212,p,212
319195,5l4k,P,O00232,1,456,1,456,1,456,307,n,307
319195,5l4k,P,O00232,1,456,1,456,1,456,368,p,368
319195,5l4k,P,O00232,1,456,1,456,1,456,448,p,448
